In [ ]:
import pandas as pd
import PrepDataHelper as helper
import pymc3 as pm
import pymc3_models as models

In [ ]:
physio_file = 'full_logfile_scr_3.csv'
df = pd.read_csv(physio_file)

#keep only spider (3) and social conditions (2)
df =df.loc[df.video_condition.isin([2,3])]
df = df.drop(df.loc[df.rt_fear>df.rt_fear.mean()+3*df.rt_fear.std()].index)
# recoded for negative valence is closer to one
df['resp_valence']= 1- df['resp_valence']
df['hp_change_video']=df['video_hp']-df['base_ECG']
df['scr_change_video']=df['video_scr']-df['base_scr']
df = helper.TakeAwayGroupsLessThenX(df, min_groups=12)

In [ ]:
standardized_fear_scr_hp  = df[['hp_change_video','participant_num','scr_change_video','resp_fear']].groupby('participant_num').transform(lambda x: (x - x.mean()) / x.std())
standardized_df = pd.concat([standardized_fear_scr_hp, df[['participant_num','video_condition']]], axis=1)
standardized_df = standardized_df.loc[df.video_condition.isin([1,2,3])]
full_df = standardized_df
full_df['video_condition']=full_df['video_condition']-1

## Individual Models

In [ ]:
y = 'resp_fear'
situation_subj_model_scr = models.RegressOnePredictor_SituationSubjectHierarchical(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y,
    'scr_change_video')

subj_hierarchical_model_scr = models.RegressOnePredictor_SubjectHierarchical(
    full_df[['scr_change_video','resp_fear','participant_num']],
    y,
    'scr_change_video')

flat_model_scr = models.RegressOnePredictor(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y,
    'scr_change_video')

situation_model_scr = models.RegressOnePredictor_Situation(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y,
    'scr_change_video')

participant_model_scr = models.RegressOnePredictor_Idiographic(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y,
    'scr_change_video')

independent_situation_participant_scr =models.RegressOnePredictor_SituationSubjectIndependent(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y,
    'scr_change_video')

meanmodel_scr = models.MeanModel(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y)

In [ ]:
n_samples = 10000
flat_model_trace_scr = models.SampleModel(flat_model_scr, n_samples)
situation_model_trace_scr = models.SampleModel(situation_model_scr, n_samples)
subj_hierarchical_model_trace_scr = models.SampleModel(subj_hierarchical_model_scr, n_samples)
situation_subj_model_trace_scr = models.SampleModel(situation_subj_model_scr, n_samples)
participant_model_trace_scr = models.SampleModel(participant_model_scr, n_samples)
participant_situation_model_trace_scr = models.SampleModel(independent_situation_participant_scr, n_samples)
mean_model_trace_scr =  models.SampleModel(meanmodel_scr, n_samples)

pm.plot_posterior(subj_hierarchical_model_trace_scr, var_names = ['mu_scr_change_video'])
pm.plot_posterior(situation_subj_model_trace_scr, var_names = ['mu_scr_change_video'])

scr_compare = pm.compare({
        'flat_model_scr': flat_model_trace_scr,
        'subj_hierarchical_model_scr':subj_hierarchical_model_trace_scr,
        'situation_subj_model_scr':situation_subj_model_trace_scr,
        'situation_model_scr': situation_model_trace_scr,
        'participant_model_scr':participant_model_trace_scr,
        'independent_situation_participant_scr':participant_situation_model_trace_scr,
        'mean_model':mean_model_trace_scr
    }, 
        ic='LOO',scale='negative_log')
scr_compare.weight = helper.CalcAkaikeWeights(scr_compare.d_loo)
scr_compare.to_csv('spidersocial_scr_loo.csv')

In [ ]:
scr_compare.weight = helper.CalcAkaikeWeights(scr_compare.d_loo)
scr_compare.to_csv('spidersocial_scr_loo.csv')

In [ ]:
y = 'resp_fear'
situation_subj_model_hp = models.RegressOnePredictor_SituationSubjectHierarchical(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y,
    'hp_change_video')

subject_hierarchical_model_hp = models.RegressOnePredictor_SubjectHierarchical(
    full_df[['hp_change_video','resp_fear','participant_num']],
    y,
    'hp_change_video')

flat_model_hp = models.RegressOnePredictor(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y,
    'hp_change_video')

situation_model_hp = models.RegressOnePredictor_Situation(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y,
    'hp_change_video')

participant_model_hp = models.RegressOnePredictor_Idiographic(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y,
    'hp_change_video')

independent_situation_participant_hp =models.RegressOnePredictor_SituationSubjectIndependent(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y,
    'hp_change_video')

meanmodel_hp = models.MeanModel(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y)



In [ ]:
n_samples = 1000
participant_model_trace_hp = models.SampleModel(participant_model_hp, n_samples)
participant_situation_model_trace_hp = models.SampleModel(independent_situation_participant_hp, n_samples)
mean_model_trace_hp =  models.SampleModel(meanmodel_hp, n_samples)

flat_model_trace_hp = models.SampleModel(flat_model_hp, n_samples)
situation_model_trace_hp = models.SampleModel(situation_model_hp, n_samples)
subject_hierarchical_model_trace_hp = models.SampleModel(subject_hierarchical_model_hp, n_samples)
situation_subj_model_trace_hp = models.SampleModel(situation_subj_model_hp, n_samples)


pm.plot_posterior(subject_hierarchical_model_trace_hp, var_names = ['mu_hp_change_video'])
pm.plot_posterior(situation_subj_model_trace_hp, var_names =['mu_hp_change_video'])

hp_compare = pm.compare({
    'General': flat_model_trace_hp,
    'Subject Hierarchical':subject_hierarchical_model_trace_hp,
    'Situation Hierarchical':situation_subj_model_trace_hp,
    'Situation':situation_model_trace_hp,
    'participant_model_hp':participant_model_trace_hp,
    'independent_situation_participant_hp':participant_situation_model_trace_hp,
    'mean_model':mean_model_trace_hp
}, ic='LOO',scale='Negative_log')

hp_compare.weight = helper.CalcAkaikeWeights(hp_compare.d_loo)
hp_compare.to_csv('spidersocial_hp_loo.csv')